In [1]:
import h5py
import ssl
import numpy as np
import sys

In [3]:
ssl._create_default_https_context = ssl._create_unverified_context
f = h5py.File('BrainWaveData.mat', 'r')
dataset = list(f["power_struct"]["highgamma"]["powspctrm"])
dataset = dataset[2000:4000]
labelsset = np.array(f["power_struct"]["beh"]["gambles"])
dataset = np.array(dataset)
dataset = np.transpose(dataset, (2,1,0))
print(labelsset[0])
print(len(labelsset[0]))
print(f)

[1 0 0 1 0 1 1 0 0 1 0 1 0 1 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 1 0
 1 0 0 0 1 1 0 1 1 1 0 1 0 1 1 0 0 1 0 1 0 0 1 1 0 0 0 0 1 1 0 1 1 1 0 1 0
 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 1 0 0 0 0 1 1 0 0 0 0 1 0
 1 0 0 0 1 0 1 0 1 0 0 1 1 0 1 0 1 1 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 0 1 1
 1 1 1 0 0 0 0 1 1 1 1 1 0 0 0 0 1 0 1 0 1 0 1 0]
172
<HDF5 file "BrainWaveData.mat" (mode r)>


In [5]:
regionset = np.array(list(f["power_struct"]["anat"]["ROIs"][0]))
regionset = [''.join(chr(i[0]) for i in f[elec]) for elec in regionset]
print(regionset)


['Amygdala', 'Amygdala', 'Amygdala', 'white_matter', 'white_matter', 'TemporalLobe', 'TemporalLobe', 'TemporalLobe', 'Cingulate', 'Cingulate', 'white_matter', 'white_matter', 'LPFC', 'LPFC', 'LPFC', 'Hippocampus', 'white_matter', 'white_matter', 'TemporalLobe', 'TemporalLobe', 'TemporalLobe', 'OFC', 'OFC', 'Cingulate', 'Cingulate', 'Cingulate', 'Cingulate', 'white_matter', 'LPFC', 'Hippocampus', 'Hippocampus', 'Hippocampus', 'white_matter', 'Hippocampus', 'Hippocampus', 'white_matter', 'TemporalLobe', 'Amygdala', 'Amygdala', 'white_matter', 'white_matter', 'TemporalLobe', 'TemporalLobe', 'TemporalLobe', 'Cingulate', 'Cingulate', 'white_matter', 'white_matter', 'LPFC', 'LPFC', 'LPFC', 'LPFC', 'white_matter', 'white_matter', 'TemporalLobe', 'TemporalLobe', 'TemporalLobe', 'OFC', 'white_matter', 'white_matter', 'white_matter', 'white_matter', 'LPFC', 'LPFC', 'LPFC', 'white_matter', 'white_matter', 'white_matter', 'white_matter', 'TemporalLobe', 'TemporalLobe']


In [7]:
data_split_by_region = [[trial[np.where(np.array(regionset) == label)] for label in np.unique(regionset)] for trial in dataset]

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn import decomposition

pca = decomposition.PCA(n_components=1)
pca_data = [[[pca.fit_transform(StandardScaler().fit_transform(d.T))] for d in data]for data in data_split_by_region]
pca_data = np.squeeze(np.array(pca_data))


In [9]:
print(pca_data.shape)
print(pca_data)

(172, 7, 2000)
[[[ 0.13120964 -0.01489183 -0.18633628 ... -0.21669862 -0.09101249
   -0.04087932]
  [-1.06823446 -1.08308446 -0.97929318 ... -0.33112804 -0.57814895
   -0.84456703]
  [-0.14784028 -0.41036582 -0.60156443 ... -1.32370598 -1.37064948
   -1.4582452 ]
  ...
  [ 1.08431604  0.65275885  0.15222983 ... -1.03335078 -1.07079616
   -1.06182938]
  [ 2.5548266   2.2812855   1.80838349 ... -0.5991005  -0.44051495
   -0.26035834]
  [ 2.62905122  2.37185866  1.98219852 ... -0.23266641 -0.42109889
   -0.73822345]]

 [[-0.39783472 -0.37470919 -0.40707894 ... -0.56147849 -0.7955811
   -0.99781944]
  [-1.68660425 -1.35722157 -0.66443331 ... -0.23934468 -0.28060189
   -0.41420708]
  [-1.27700245 -1.0524507  -0.7988975  ...  1.07833368  1.30417686
    1.36542047]
  ...
  [-1.62669687 -1.63570906 -1.55876037 ... -0.05269747 -0.21219204
   -0.44113449]
  [-0.94109129 -0.71230028 -0.58529436 ... -0.88299769 -1.14137025
   -1.38916801]
  [-1.67231011 -1.60270798 -1.4513983  ... -0.15846861  0.0

In [10]:
binned = [[(np.histogram(region, 5, weights=region)[0] /
             np.histogram(region, 5)[0]) for region in trial] for trial in pca_data]
binned = np.array(binned)
print(binned.shape)


(172, 7, 5)


In [11]:
parameters = np.array([trial.flatten() for trial in binned])
print(parameters.shape)


(172, 35)


In [12]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(parameters, np.squeeze(labelsset), test_size=0.3, random_state=42)


rf = RandomForestRegressor(n_estimators = 1000)
rf.fit(X_train, y_train);
#print(rf.fit)
truth = np.squeeze(labelsset)

#print(truth.reshape(-1,1))

y_pred = rf.predict(np.squeeze(X_test))
#print(rf)
#print(y_pred)


for i in range(len(y_pred)):
    if y_pred[i] > .5:
        y_pred[i] = 1
        y_pred[i] = int(y_pred[i])
    else:
        y_pred[i] = 0
        y_pred[i] = int(y_pred[i])
print(len(y_pred))

truth = np.transpose(labelsset)

mae = mean_absolute_error(y_pred, y_test)

print("Accuracy:", (1-mae))
##find accuracy

52
Accuracy: 0.6538461538461539


In [13]:
importance = np.reshape(rf.feature_importances_, (7,5))
print(np.sum(importance, 1))
print(np.sum(importance, 0))
#200 ms bin size importance
print(np.unique(regionset))
#assess importance of parameters

[0.11572667 0.12722237 0.10420704 0.11382204 0.14229516 0.25669622
 0.14003049]
[0.26929568 0.16965277 0.15234427 0.17455016 0.23415712]
['Amygdala' 'Cingulate' 'Hippocampus' 'LPFC' 'OFC' 'TemporalLobe'
 'white_matter']
